# Homework pandas

<table align="left">
    <tr>
    <td><a href="https://colab.research.google.com/github/airnandez/numpandas/blob/master/exam/2020-exam.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a></td>
    <td><a href="https://mybinder.org/v2/gh/airnandez/numpandas/master?filepath=exam%2F2020-exam.ipynb">
  <img src="https://mybinder.org/badge_logo.svg" alt="Launch Binder"/>
</a></td>
  </tr>
</table>

*Author: Fabio Hernandez*

*Last updated: 2020-03-19*

*Location:* https://github.com/airnandez/numpandas/exam

--------------------
## Instructions

For this excercise we will use a public dataset titled **"Demandes de valeurs foncières géolocalisées"** available [here](https://www.data.gouv.fr/fr/datasets/demandes-de-valeurs-foncieres-geolocalisees/). This dataset contains information about registered real state transactions (_mutations immobilières_) in France over several years. There is a file per year. The structure of the files and the semantics of each column are documented at its source.

For your convenience, this notebook is prepared with code for downloading the dataset from its source, loading it into memory as a **pandas** dataframe and with some cleaning and helper functions. Your mission is execute the provided cells and to write the code to answer the questions below.

You must not modify the code provided. You must provide code for answering the questions, following the instructions for each one of them.

When you have finished, please save your notebook in the form of a `.ipynb` file and send it by e-mail to your instructor according to the indications you received by e-mail.

---------------------
## Dependencies

In [ ]:
import datetime
import os
import glob

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.__version__

In [ ]:
import numpy as np
np.__version__

------
## Download the dataset

Define a helper function for downloading data to a local file:

In [ ]:
import requests
import shutil

def download_file(url, path):
    with requests.get(url, stream=True) as r:
        with open(path, 'wb') as f:
            shutil.copyfileobj(r.raw, f)
    return

Download the data files, one per year, for the period 2014-2019, both inclusive. We store the downloaded data in the directory `../data` relative to the location of this notebook. If a file has been already been downloaded, don't download it again. The total amount of data to download is about 400 MB.

In [ ]:
# Create destination directory
os.makedirs(os.path.join('..', 'data'), exist_ok=True)

# Download files
data_source = "https://cadastre.data.gouv.fr/data/etalab-dvf/latest/csv"

for year in range(2014, 2020):
    # Build the URL and the destination file path
    url = f'{data_source}/{year}/full.csv.gz'
    path = os.path.join('..', 'data', f'{year}-mutations-immobilieres.csv.gz')
    
    # If file already exists don't download it again
    if not os.path.isfile(path) :
        print(f'downloading {url} to {path}')
        download_file(url, path)

Check what files we have for our analysis:

In [ ]:
file_paths = glob.glob(os.path.join('..', 'data', '*-mutations-immobilieres.csv.gz'))
print('\n'.join(f for f in file_paths))

---------------------
## Load the dataset

Load the dataset (i.e. all the files `../data/*-mutations-immobilieres.csv.gz`) to a **pandas** dataframe. Here we select the columns we want to load. The information about the format and contents of each column is available [here](https://www.data.gouv.fr/fr/datasets/demandes-de-valeurs-foncieres-geolocalisees/). Please make sure you are familiar with that information which you will need for analysing the data:

In [ ]:
# These are the names of the columns present in the source files.
# We are not interested in analysing the commented columns, so we don't tell
# pandas to not load them
columns = (
    'id_mutation',
    'date_mutation',
    'numero_disposition',
    'nature_mutation',
    'valeur_fonciere',
    'adresse_numero',
    'adresse_suffixe',
    'adresse_nom_voie',
    'adresse_code_voie',
    'code_postal',
    'code_commune',
    'nom_commune',
    'code_departement',
#   'ancien_code_commune',
#   'ancien_nom_commune',
#   'id_parcelle',
#   'ancien_id_parcelle',
#   'numero_volume',
    'lot1_numero',
    'lot1_surface_carrez',
    'lot2_numero',
    'lot2_surface_carrez',
    'lot3_numero',
    'lot3_surface_carrez',
    'lot4_numero',
    'lot4_surface_carrez',
    'lot5_numero',
    'lot5_surface_carrez',
    'nombre_lots',
    'code_type_local',
    'type_local',
    'surface_reelle_bati',
    'nombre_pieces_principales',
#   'code_nature_culture',
    'nature_culture',
#   'code_nature_culture_speciale',
#   'nature_culture_speciale',
    'surface_terrain',
#   'longitude',
#   'latitude'
)

# These are the types we want pandas to use for each column
column_types = {
    'id_mutation': object,
    'adresse_suffixe': str,
    'adresse_numero': str,
    'adresse_suffixe': str,
    'adresse_nom_voie': str,
    'adresse_code_voie': str,
    'code_postal': str,
    'code_commune': str,
    'code_departement': str,
    'ancien_code_commune': str,
    'ancien_nom_commune': str,
    'id_parcelle': str,
    'ancien_id_parcelle': str,
    'lot1_numero': str,
    'lot2_numero': str,
    'lot3_numero': str,
    'lot4_numero': str,
    'lot5_numero': str,
    'code_type_local': str,
    'type_local': str,
}

In [ ]:
# Explicitly delete our existing dataframe, if any
try:
    del df
except NameError:
    pass

file_paths = glob.glob(os.path.join('..', 'data', '*-mutations-immobilieres.csv.gz'))
df = pd.DataFrame()
for path in sorted(file_paths):
    print(f'Loading {path}')
    df = df.append(pd.read_csv(path, usecols=columns, dtype=column_types, parse_dates=['date_mutation']))

In [ ]:
# Inspect the dimensions of the dataframe
rows, columns = df.shape
print(f'This dataframe has {rows:,} rows and {columns:,} columns')

In [ ]:
df.head(10)

### WARNING:

Please note that there may be several rows for the same transaction. All the rows part of a single transaction have the same identifier (i.e. the same value) in the `id_mutation` column as well as the same value in the column `valeur_fonciere`. For instance, there are two rows with the value `2014-2` in the `id_mutation` column:

In [ ]:
df[df['id_mutation'] == '2014-2']

## Inspect the dataset

Let's see what **kind of transactions** are encoded in these records:

In [ ]:
print('\n'.join(df['nature_mutation'].unique()))

And what **kind of properties** are these transactions about:

In [ ]:
for t in df['type_local'].unique():
    print(t)

### Values for filters
Here we define some convenient constants that we can use for building masks:

In [ ]:
APPARTMENT = 'Appartement'
HOUSE      = 'Maison'
BUSINESS   = 'Local industriel. commercial ou assimilé'

-------------------
# Questions (10 points + bonus)

---------------------
## Question N° 1

### Question 1a (1 point)
How many transactions of type sale (i.e. those with value `Vente` in the column `nature_mutation`) were registered in the period covered in the dataset?

In [ ]:
# Your code here


sales_count = ...

In [ ]:
print(f'There are {sales_count:,} sales in the dataset')

### Question 1b (1 point)
How many sales were registered for each kind of property (i.e. `Maison`, `Dépendance`, `Appartement` and `Local industriel`) in the whole period?

In [ ]:
# Your code here


house_count    = ...
appt_count     = ...
business_count = ...

In [ ]:
# Determine the period covered in the dataset
start_date, end_date = df['date_mutation'].min(), df['date_mutation'].max()

# Compute the percentage of sales per kind of object
house_pct    = 100.0 * (house_count/sales_count)
appt_pct     = 100.0 * (appt_count/sales_count)
business_pct = 100.0 * (business_count/sales_count)

# Print the report
print(f'Period covered: from {start_date:%Y-%m-%d} to {end_date:%Y-%m-%d}:')
print(f'          total:  {sales_count:>10,} sales')
print(f'         houses:  {house_count:>10,} ({house_pct:>2.0f}%)')
print(f'    appartments:  {appt_count:>10,} ({appt_pct:>2.0f}%)')
print(f'       business:  {business_count:>10,} ({business_pct:>2.0f}%)')

### Question 1c (2 points)
What is the total amount of money (in million €) involved in those sales? Please remember that there may be several rows for a single transaction and within a single transaction each row has the same value in the column `valeur_fonciere`. You may want to consider grouping all the rows for the same transaction.

In [ ]:
# Your code here

sales_in_million_euros = ...

In [ ]:
print(f'The total amount of money in sales was {sales_in_million_euros:,.0f} million €')

-----------
## Question N° 2

### Question 2a (3 points)
Your client, a big international corporation, is looking to purchase a property for installing a retail store in the Av. des Champs Elysées, in Paris. They hire you to provide an estimation of the necessary budget to purchase a property based on the data recorded in this dataset. You should only consider transactions involving business properties with a surface bigger than 300 m², 

In [ ]:
# First, build a view with the relevant data





In [ ]:
# Select and aggregate the relevant data



# Compute the average of the cost per square meter for each transaction
mean_cost_per_sq_meter = ...

In [ ]:
print(f'The average observed cost per square meter, for business bigger than 300 m² is {mean_cost_per_sq_meter:,.0f} €')

### Question 2b (3 points)

Your customer also wants to know how much money was needed for the most expensive transaction and the address of the property. Can you provide them that information?

In [ ]:
# Your code here


max_cost = ...
address  = ...

In [ ]:
print(f'The cost of the biggest sale transaction was {max_cost/1_000_000:,.0f} m€ for a property located at {address}')

### Question 2c (bonus: 1 point)
Can you tell what store is now located at the address found in your answer for question 2b?